# Preparo

In [ ]:
#importa as bibliotecas necessárias para cada seção

import pandas as pd
import numpy as np
import datetime

#Biblioteca necessária para o download de arquivos
from google.colab import files


#Importa recursos do Colab para troca de recursos com o Drive
!pip install --upgrade gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


#Define o dia dos relatórios a serem gerados, em formato de string
dia_str = input('Dia do relatório (aaaa-mm-dd): ')




Dia do relatório (aaaa-mm-dd): 2022-02-14


# Produtividade hora a hora

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Entrada']]
dimens = dimens[dimens['Login'].notna()]

#Lê o arquivo em csv com os detalhes de cada atendimento
actions = pd.read_csv('actions_done.csv', sep = ',')

#Converte os dados das colunas de início e término dos jobs em 'datetime'
actions['Action Ended At Time'] = pd.to_datetime(actions['Action Ended At Time'])
actions['Action Started At Time'] = pd.to_datetime(actions['Action Started At Time'])

#Filtros para os jobs iniciados ou finalizados no dia desejado
filtro_dia_inicio = actions['Action Started Date'] == dia_str
filtro_dia_termino = actions['Action Ended Date'] == dia_str

#Filtra os dados gerais de acordo com o dado desejado
actions_dia = actions[filtro_dia_inicio | filtro_dia_termino]


#Inclui colunas de horas iniciais e finais
actions_dia['hora_inicio'] = actions_dia['Action Started At Time'].apply(lambda x: x.hour)
actions_dia['hora_termino'] = actions_dia['Action Ended At Time'].apply(lambda x: x.hour)

#Define os canais de atendimento
channels = ['chat', 'inbound_call', 'email']

#Cria dicionário de produtividade hora a hora (chaves = logins, valores = detalhamento dos jobs)
hora_a_hora = {}

#Cria dataframe que será utilizado ao final
produtividade = pd.DataFrame()

#Varre todos os canais de atendimento
for canal in channels:

  #Filtra os jobs por canal
  filtro_canal = actions_dia['Job Source'] == canal
  actions_canal = actions_dia[filtro_canal]

  #Varre todos os agentes (logins) da operação
  for agente in dimens['Login']:

    #Filtra os jobs por agente
    filtro_agente = actions_canal['Action Actor ID'] == agente
    df_agente = actions_canal[filtro_agente]

    #Determina as horas nas quais houve atividade
    horas = range(24)

    #Cria um dataframe com as horas nas quais houve atividade e renomeia a coluna
    df = pd.DataFrame(horas)
    df.rename(columns = {0: 'horas'}, inplace = True)

    #Coluna equivalente aos atendimentos iniciados por hora
    df_agente_inic = df_agente[df_agente['Action Started Date'] == dia_str]

    df['iniciados'] = df.horas.apply(lambda x: list(df_agente_inic.hora_inicio).count(x))

    #Status possíveis para cada atendimento
    status = ['finished', 'skipped', 'transferred']

    #Varre todos os status de finalização possíveis para cada atendimento
    for i in status:
      
      #Filtra por status de atendimento
      filtro_status = df_agente['Action Final Status'] == i

      #Cria colunas de atendimentos por hora para o status em questão
      df_agente_fin = df_agente[df_agente['Action Ended Date'] == dia_str]

      df[i] = df.horas.apply(lambda x: list(df_agente_fin[filtro_status].hora_termino).count(x))

      #Inclui colunas com o login e com o canal em questão
      df['Login'] = agente
      df['canal'] = canal
    
      #Cria chave (login) e valor para o dicionário
      hora_a_hora[str(agente) + '_' + str(canal)] = df

#Concatena todos os valores (dataframes) do dicionário
for key in hora_a_hora:
  produtividade = pd.concat([produtividade, hora_a_hora[str(key)]])

#Cria coluna com a data em questão
produtividade['Data'] = dia_str

#Unifica o dataframe final com a base utilizada
produtividade = pd.merge(produtividade, dimens, how = 'inner', on = 'Login')

#Altera os nomes de cada canal em inglês para o português
produtividade.canal = produtividade.canal.replace('chat', 'Chat').replace('inbound_call', 'Phone').replace('email', 'Email')

#Cria coluna com o total de atividades por hora
produtividade['Total'] = produtividade.iniciados + produtividade.finished + produtividade.skipped + produtividade.transferred

#Filtra e elimina as horas nas quais não houve atendimento algum
filtro_horas = produtividade.Total != 0
produtividade = produtividade[filtro_horas]

#Organiza as colunas do dataframe e classifica este
produtividade = produtividade [['Data', 'Login', 'Nome', 'Supervisor(a)', 'canal', 'horas', 'iniciados', 'finished', 'skipped', 'transferred']]
produtividade.sort_values(by= ['Nome', 'horas', 'canal'], inplace = True)

#Define a planilha (sheet) a ser utilizada
hora_a_hora_sheets = gc.open('Hora a hora/ADH_02/2022')

#Define a aba (worksheet) a ser utilizada
produtividade_hora_a_hora_page = hora_a_hora_sheets.worksheet('Produtividade_hora_a_hora')

#Cria um dataframe com os dados da aba selecionada
df_produtividade_hora_a_hora = pd.DataFrame(produtividade_hora_a_hora_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_produtividade_hora_a_hora = df_produtividade_hora_a_hora[df_produtividade_hora_a_hora.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
produtividade.columns = novo_produtividade_hora_a_hora.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_produtividade_hora_a_hora = pd.concat([novo_produtividade_hora_a_hora, produtividade])

#Atualiza a aba selecionada com o DataFrame resultante
produtividade_hora_a_hora_page.update([novo_produtividade_hora_a_hora.columns.values.tolist()] + novo_produtividade_hora_a_hora.values.tolist())

#Retoma os nomes das colunas do dataframe
produtividade.columns = ['Data', 'Login', 'Nome', 'Supervisor(a)', 'canal', 'horas', 'iniciados', 'finished', 'skipped', 'transferred']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


# ADH por jobs iniciados

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Canal', 'Entrada', 'Jornada']]
dimens = dimens[dimens['Login'].notna()]

#Define os canais de atendimento
channels = produtividade.canal.unique()

#Filtra o dataframe de produtividade para as horas com algum atendimento iniciado
filtro_iniciados = produtividade.iniciados > 0
iniciados = produtividade[filtro_iniciados]

#Cria dataframe para a visão da aderência (atendentes x horas)
adh_iniciados = pd.DataFrame()

#Varre todos os canais de atendimento
for channel in channels:

  #Filtra o canal de atendimento em questão
  filtro_canal = iniciados.canal == channel
  adh_iniciados_channel = iniciados[filtro_canal]
  
  #Remove linhas duplicadas (quanto ao login)
  adh_iniciados_channel.drop_duplicates(subset = 'Login', inplace = True)

  #Varre todas as horas
  for i in range(24):

    #Filtra a hora em questão
    filtro_hora = iniciados.horas == i
    adh_iniciados_horas = iniciados[filtro_hora]

    #Define se houve atendimento iniciado ("Ok") ou não na hora em questão por determinado atendente
    adh_iniciados_channel[i] = adh_iniciados_channel['Login'].apply(lambda x: 'Ok' if adh_iniciados_horas[adh_iniciados_horas['Login'] == x].shape[0] > 0 else '')

  #Unifica o dataframe final com a base utilizada e organiza as colunas resultantes
  adh_iniciados_channel = pd.merge(adh_iniciados_channel, dimens, how = 'inner', on = ['Login', 'Nome', 'Supervisor(a)'])[['Data', 'Login','canal', 'Nome', 'Supervisor(a)', 'Jornada', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]
  adh_iniciados_channel = adh_iniciados_channel[['Data','canal', 'Nome', 'Supervisor(a)', 'Jornada', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]

  #Unifica o dataframe resultante ao prévio para resultado final
  adh_iniciados = pd.concat([adh_iniciados, adh_iniciados_channel])

#Define a planilha (sheet) a ser utilizada
hora_a_hora_sheets = gc.open('Hora a hora/ADH_02/2022')

#Define a aba (worksheet) a ser utilizada
ahd_iniciados_page = hora_a_hora_sheets.worksheet('ADH_Iniciados')


#Cria um dataframe com os dados da aba selecionada
df_ahd_iniciados = pd.DataFrame(ahd_iniciados_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_ahd_iniciados = df_ahd_iniciados[df_ahd_iniciados.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
adh_iniciados.columns = novo_df_ahd_iniciados.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_ahd_iniciados = pd.concat([novo_df_ahd_iniciados, adh_iniciados])

#Atualiza a aba selecionada com o DataFrame resultante
ahd_iniciados_page.update([novo_df_ahd_iniciados.columns.values.tolist()] + novo_df_ahd_iniciados.values.tolist())

#Retoma os nomes das colunas do dataframe
adh_iniciados.columns = ['Data','canal', 'Nome', 'Supervisor(a)', 'Jornada', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
actions['Action Started Date'].unique()

array(['2022-02-15', '2022-02-14', '2022-02-13', '2022-02-12',
       '2022-02-11', '2022-02-10', '2022-02-09', '2022-02-08',
       '2022-02-07'], dtype=object)

In [ ]:
dia_str

'2022-02-14'

# ADH por jobs finalizados

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Canal', 'Entrada', 'Jornada']]
dimens = dimens[dimens['Login'].notna()]

#Define os canais de atendimento
channels = produtividade.canal.unique()

#Filtra o dataframe de produtividade para as horas com algum atendimento iniciado
filtro_finished = produtividade.finished > 0
finished = produtividade[filtro_finished]

#Cria dataframe para a visão da aderência (atendentes x horas)
adh_finished = pd.DataFrame()

#Varre todos os canais de atendimento
for channel in channels:

  #Filtra o canal de atendimento em questão
  filtro_canal = finished.canal == channel
  adh_finished_channel = finished[filtro_canal]
  
  #Remove linhas duplicadas (quanto ao login)
  adh_finished_channel.drop_duplicates(subset = 'Login', inplace = True)

  #Varre todas as horas
  for i in range(24):

    #Filtra a hora em questão
    filtro_hora = finished.horas == i
    adh_finished_horas = finished[filtro_hora]

    #Define se houve atendimento iniciado ("Ok") ou não na hora em questão por determinado atendente
    adh_finished_channel[i] = adh_finished_channel['Login'].apply(lambda x: 'Ok' if adh_finished_horas[adh_finished_horas['Login'] == x].shape[0] > 0 else '')

  #Unifica o dataframe final com a base utilizada e organiza as colunas resultantes
  adh_finished_channel = pd.merge(adh_finished_channel, dimens, how = 'inner', on = ['Login', 'Nome', 'Supervisor(a)'])[['Data', 'Login','canal', 'Nome', 'Supervisor(a)', 'Jornada', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]
  adh_finished_channel = adh_finished_channel[['Data','canal', 'Nome', 'Supervisor(a)', 'Jornada', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]

  #Unifica o dataframe resultante ao prévio para resultado final
  adh_finished = pd.concat([adh_finished, adh_finished_channel])

#Define a planilha (sheet) a ser utilizada
hora_a_hora_sheets = gc.open('Hora a hora/ADH_02/2022')

#Define a aba (worksheet) a ser utilizada
ahd_finalizados_page = hora_a_hora_sheets.worksheet('ADH_Finalizados')

#Cria um dataframe com os dados da aba selecionada
df_ahd_finalizados = pd.DataFrame(ahd_finalizados_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_ahd_finalizados = df_ahd_finalizados[df_ahd_finalizados.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
adh_finished.columns = novo_df_ahd_finalizados.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_ahd_finalizados = pd.concat([novo_df_ahd_finalizados, adh_finished])

#Atualiza a aba selecionada com o DataFrame resultante
ahd_finalizados_page.update([novo_df_ahd_finalizados.columns.values.tolist()] + novo_df_ahd_finalizados.values.tolist())

#Retoma os nomes das colunas do dataframe
adh_finished.columns = ['Data','canal', 'Nome', 'Supervisor(a)', 'Jornada', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#Sources IDs de jobs skipados (geral) ou transferidos (Phone)

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Entrada']]
dimens = dimens[dimens['Login'].notna()]

#Cria um filtro para os jobs com o status de 'skipped' realizados no dia em questão
filtro_skip = ((actions['Action Final Status'] == 'skipped') & filtro_dia_termino)

#Cria um filtro para os jobs com o status de 'transferred' realizados no dia em questão para o canal de Phone
filtro_transfer_phone = (((actions['Action Final Status'] == 'transferred') & (actions['Job Source'] == 'inbound_call')) & filtro_dia_termino)

#Filtra os jobs de acordo com os filtros criados acima
IDs_skip_transfer = actions[filtro_skip | filtro_transfer_phone]

#Seleciona as colunas úteis para a tratativa dos dados
IDs_skip_transfer = IDs_skip_transfer[['Action Ended Date', 'Action Actor ID', 'Job Source','Action Started At Time', 'Action Ended At Time', 'Action Final Status', 'Source ID']]

#Renomeia as colunas para o português
IDs_skip_transfer.rename(columns = {'Action Ended Date': 'Data', 'Action Actor ID': 'Login', 'Job Source': 'canal','Action Started At Time' : 'inicio', 'Action Ended At Time': 'termino', 'Action Final Status': 'status', 'Source ID' : 'ID'}, inplace = True)

#Substitui os termos em inglês referentes aos canais de Chat, Phone e Email
IDs_skip_transfer.canal = IDs_skip_transfer.canal.replace('chat', 'Chat').replace('inbound_call', 'Phone').replace('email', 'Email')

#Unifica os dataframes de dimensionamento e de skips
IDs_skip_transfer = pd.merge(IDs_skip_transfer, dimens, how = 'inner', on = 'Login')

#Organiza as colunas do dataframe resultante
IDs_skip_transfer = IDs_skip_transfer[['Data', 'canal', 'Nome', 'Login', 'Supervisor(a)', 'inicio', 'termino', 'status', 'ID']]

#Classifica o dataframe
IDs_skip_transfer.sort_values(by= ['status', 'termino', 'Nome'], ascending = True, inplace = True)

#Reescreve os valores das colunas "inicio" e "termino" apenas com as horas
IDs_skip_transfer.inicio = IDs_skip_transfer.inicio.apply(lambda x: str(x.time()))
IDs_skip_transfer.termino = IDs_skip_transfer.termino.apply(lambda x: str(x.time()))

#Define a planilha (sheet) a ser utilizada
skip_sheets = gc.open('Skip/Transfer_02/2022')

#Define a aba (worksheet) a ser utilizada
ids_skip_page = skip_sheets.worksheet('IDs_Skip')

#Cria um dataframe com os dados da aba selecionada
df_ids_skip = pd.DataFrame(ids_skip_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_ids_skip = df_ids_skip[df_ids_skip.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
IDs_skip_transfer.columns = novo_df_ids_skip.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_ids_skip = pd.concat([novo_df_ids_skip, IDs_skip_transfer])

#Atualiza a aba selecionada com o DataFrame resultante
ids_skip_page.update([novo_df_ids_skip.columns.values.tolist()] + novo_df_ids_skip.values.tolist())

{'spreadsheetId': '1LNSuv0fDOqhQhABhfkNb-l3VEJfyyLDEe8a-uvZ2MLs',
 'updatedCells': 27531,
 'updatedColumns': 9,
 'updatedRange': 'IDs_Skip!A1:I3059',
 'updatedRows': 3059}

# Skip/Transfer detalhado

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Entrada']]
dimens = dimens[dimens['Login'].notna()]

#Agrupa os dados do dataframe de produtividade de acordo com as ocorrências de "skipped" e "transferred"
skip_detalhado = pd.DataFrame(produtividade.groupby(['Data', 'canal','Nome', 'Supervisor(a)'])['skipped', 'transferred'].sum()).reset_index()

#Classifica o dataframe resultante
skip_detalhado.sort_values(by= ['skipped', 'transferred', 'Nome'], ascending = False, inplace = True)

#Define a planilha (sheet) a ser utilizada
skip_sheets = gc.open('Skip/Transfer_02/2022')

#Define a aba (worksheet) a ser utilizada
skip_transfer_page = skip_sheets.worksheet('Análise Skip/Transfer')

#Cria um dataframe com os dados da aba selecionada
df_skip_transfer = pd.DataFrame(skip_transfer_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_skip_transfer = df_skip_transfer[df_skip_transfer.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
skip_detalhado.columns = novo_df_skip_transfer.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_skip_transfer = pd.concat([novo_df_skip_transfer, skip_detalhado])

#Atualiza a aba selecionada com o DataFrame resultante
skip_transfer_page .update([novo_df_skip_transfer.columns.values.tolist()] + novo_df_skip_transfer.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


{'spreadsheetId': '1LNSuv0fDOqhQhABhfkNb-l3VEJfyyLDEe8a-uvZ2MLs',
 'updatedCells': 19560,
 'updatedColumns': 6,
 'updatedRange': "'Análise Skip/Transfer'!A1:F3260",
 'updatedRows': 3260}

# Sources IDs de jobs que retornaram para Collections

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Entrada']]
dimens = dimens[dimens['Login'].notna()]

#Lê o arquivo em csv com os detalhes de cada atendimento
actions = pd.read_csv('actions_done.csv', sep = ',')

#Filtros para os jobs iniciados ou finalizados no dia desejado
filtro_dia_inicio = actions['Action Started Date'] == dia_str
filtro_dia_termino = actions['Action Ended Date'] == dia_str

#Filtra o arquivo com os filtros criados acima
actions = actions[filtro_dia_inicio | filtro_dia_termino]

#Agrupa os dados de acordo com a ocorrência de cada status por Source ID
u = pd.DataFrame(actions.groupby('Source ID')['Action Final Status'].count()).reset_index()

#Filtra u para ocorrências de status maiores que 1
u = u[u['Action Final Status'] > 1]

#Cria uma lista com os elementos únicos de u
filtro_ID = u['Source ID'].unique()

#Filtra actions para apenas as linhas que contém os elementos de Filtro_ID
actions = actions[actions['Source ID'].isin(filtro_ID)]

#Filtra actions para jobs que tiveram status diferente de "finished"
actions = actions[actions['Action Final Status'] != 'finished']

#Classifica o dataframe resultante
actions.sort_values(by=['Source ID', 'Action Started At Time'], inplace = True)

#Cria um dataframe vazio
jobs_repetidos = pd.DataFrame()

#Percorre cada elemento do Filtro_ID
for id in filtro_ID:

  #Seleciona até a penúltima linha de um dataframe que contém somente o mesmo Source ID
  job_repetido = actions[actions['Source ID'] == id][0:-1]

  #Concatena o dataframe resultante junto com o dataframe vazio criado anteriormente
  jobs_repetidos = pd.concat([jobs_repetidos, job_repetido])

#Renomeia a coluna de identificação do atendente
jobs_repetidos.rename(columns = {'Action Actor ID' : 'Login'}, inplace = True)

#Substitui os termos em inglês referentes aos canais de Chat, Phone e Email
jobs_repetidos['Job Source'] = jobs_repetidos['Job Source'].replace('chat', 'Chat').replace('inbound_call', 'Phone').replace('email', 'Email')

#Unifica os dataframes de dimensionamento e o jobs_repetidos
jobs_repetidos = pd.merge(jobs_repetidos, dimens, how = 'inner', on = 'Login') [['Action Ended Date', 'Job Source', 'Login', 'Nome', 'Supervisor(a)', 'Action Ended At Time', 'Action Final Status', 'Source ID']]

#Converte a coluna de início do job para datetime
jobs_repetidos['Action Ended At Time'] = pd.to_datetime(jobs_repetidos['Action Ended At Time'])

#Reescreve os valores da coluna de inicio apenas com a hora dos mesmos
jobs_repetidos['Action Ended At Time'] = jobs_repetidos['Action Ended At Time'].apply(lambda x: str(x.time()))

#Define a planilha (sheet) a ser utilizada
jobs_repetidos_sheets = gc.open('Skip/Transfer_02/2022')

#Define a aba (worksheet) a ser utilizada
jobs_repetidos_page = jobs_repetidos_sheets.worksheet('IDs_repetidos')

#Cria um dataframe com os dados da aba selecionada
df_jobs_repetidos = pd.DataFrame(jobs_repetidos_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_jobs_repetidos = df_jobs_repetidos[df_jobs_repetidos.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
jobs_repetidos.columns = novo_df_jobs_repetidos.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_jobs_repetidos = pd.concat([novo_df_jobs_repetidos, jobs_repetidos])

#Atualiza a aba selecionada com o DataFrame resultante
jobs_repetidos_page.update([novo_df_jobs_repetidos.columns.values.tolist()] + novo_df_jobs_repetidos.values.tolist())

{'spreadsheetId': '1LNSuv0fDOqhQhABhfkNb-l3VEJfyyLDEe8a-uvZ2MLs',
 'updatedCells': 3088,
 'updatedColumns': 8,
 'updatedRange': 'IDs_repetidos!A1:H386',
 'updatedRows': 386}

# Análise CSAT mensal

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Canal']]
dimens = dimens[dimens['Login'].notna()]

#Lê o arquivo em csv com as avaliações dos atendimentos pelos clientes
csat = pd.read_csv('csat.csv')

#Cria um dataframe inicialmente apenas com os logins do dimensionamento
csat_detalhado = pd.DataFrame(dimens['Login'])

#Cria uma nova coluna cujos dados sao o número de notas 1 que um determinado login recebeu 
csat_detalhado['nota_1'] = csat_detalhado['Login'].apply(lambda x: csat[(csat['Action Actor ID'] == x) & (csat['Customer Rating'] == 1)].shape[0])

#Cria uma nova coluna cujos dados sao o número de notas 4 ou 5 que um determinado login recebeu 
csat_detalhado['notas_4_ou_5'] = csat_detalhado['Login'].apply(lambda x: csat[(csat['Action Actor ID'] == x) & (csat['Customer Rating'] >= 4)].shape[0])

#Cria uma nova coluna cujos dados sao o número total de notas que um determinado login recebeu 
csat_detalhado['Total'] = csat_detalhado['Login'].apply(lambda x: list(csat['Action Actor ID']).count(x))

#Cria uma nova coluna cujos dados sao o percentual de notas 1 que um determinado login recebeu 
csat_detalhado['%_1'] = csat_detalhado.nota_1 / csat_detalhado.Total

#Cria uma nova coluna cujos dados sao o percentual de notas 4 ou 5 que um determinado login recebeu 
csat_detalhado['%_4_ou_5'] = csat_detalhado.notas_4_ou_5 / csat_detalhado.Total

#Unifica os dataframes "csat_detalhado" e "dimens" com base nos logins
csat_detalhado = pd.merge(csat_detalhado, dimens, how = 'inner', on ='Login')

#Seleciona as colunas relevantes do dataframe resultante
csat_detalhado = csat_detalhado[['Login', 'Canal', 'Nome', 'Supervisor(a)', 'nota_1',  'notas_4_ou_5', 'Total', '%_1', '%_4_ou_5']]

#Classifica o dataframe resultante
csat_detalhado.sort_values(by= ['%_4_ou_5', 'Total', 'Nome'], ascending = False, inplace = True)

#Define a planilha (sheet) a ser utilizada
csat_sheets = gc.open('CSAT_02/2022')

#Define a aba (worksheet) a ser utilizada
csat_page = csat_sheets.worksheet('Análise CSAT')

#Cria um dataframe com os dados da aba selecionada
df_csat = pd.DataFrame(csat_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_csat = df_csat[df_csat.Agente == '']

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
csat_detalhado.columns = novo_df_csat.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_csat = pd.concat([novo_df_csat, csat_detalhado])

#Preenche as células vazias com "0"
novo_df_csat = novo_df_csat.fillna(0)

#Atualiza a aba selecionada com o DataFrame resultante
csat_page.update([novo_df_csat.columns.values.tolist()] + novo_df_csat.values.tolist())

{'spreadsheetId': '1MGyoFK7bhyn6nOm90UI4ZmbokXMXM87Lhc_b2Ft2kqA',
 'updatedCells': 2313,
 'updatedColumns': 9,
 'updatedRange': "'Análise CSAT'!A1:I257",
 'updatedRows': 257}

# Intervalos sem jobs


In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)']]
dimens = dimens[dimens['Login'].notna()]

#Lê o arquivo em csv com os detalhes de cada atendimento
actions = pd.read_csv('actions_done.csv', sep = ',')

#Cria um filtro para "actions" de acordo com o dia em questão
filtro_dia = actions['Action Started Date'] == dia_str

#Filtra "actions" de acordo com o filtro criado
actions = actions[filtro_dia]

#Converte a coluna de início dos jobs para tipo datetime
actions['Action Started At Time'] = pd.to_datetime(actions['Action Started At Time'])

#Cria uma lista com os canais nos quais houve atendimento
channels = actions['Job Source'].unique()

#Cria um dataframe vazio
atencao = pd.DataFrame()

#Varre todos os canais
for channel in channels:

  #Cria um filtro para "actions" de acordo com o canal
  filtro_canal = actions['Job Source'] == channel

  #Filtra "actions" com o filtro criado e seleciona as colunas relevantes
  df_canal = actions[filtro_canal][['Job Source', 'Action Actor ID', 'Action Started At Time', 'Action Final Status']]

  #Cria uma lista com todos os logins que realizaram atendimentos
  logins = df_canal['Action Actor ID'].unique()

  #Cria um dicionário vazio
  base = {}

  #Varre todos os logins
  for login in logins:

    #Cria uma chave e um valor (dataframe) para o dicionário, com o valor sendo classificado pelo início dos jobs
    base[str(login)] = df_canal[(df_canal['Action Actor ID']) == str(login)].sort_values(by = 'Action Started At Time')

  #Cria uma condição para o canal de email
  if channel == 'email':

    #Cria um limite máximo para um intervalo sem jobs em email
    intervalo_max = pd.to_timedelta("0 days 00:30:00")

  #Cria uma condição para o canal de chat
  elif channel == 'chat':

    #Cria um limite máximo para um intervalo sem jobs em chat
    intervalo_max = pd.to_timedelta("0 days 00:40:00")
  
  #Cria uma condição para o canal de phone
  else:

    #Cria um limite máximo para um intervalo sem jobs em phone
    intervalo_max = pd.to_timedelta("0 days 00:45:00")

  #Cria um limite máximo para invervalos sem jobs no geral (considerando a CLT, intervalo mínimo de 11h entre uma jornada e outra)
  limite = pd.to_timedelta("0 days 11:00:00")

  #Varre todos as chaves e seus valores
  for login, df in base.items():

    #Cria uma lista com todos os horários de início de jobs (excluindo a primeira linha)
    lista = list(df['Action Started At Time'][1:])

    #Cria um dado do tipo datetime com a data atual (assim o intervalo será sempre maior que 11h, nao entrando na configuração final devido ao filtro seguinte)
    lista.append(pd.Timestamp('23:00:00'))

    #Cria uma coluna com o fim do intervalo sem jobs (um intervalo tem fim quando se inicia o próximo job)
    df['fim_intervalo'] = lista

    #Cria coluna com o intervalo sem jobs
    df['intervalo_sem_jobs'] = df.fim_intervalo - df['Action Started At Time']

    #Converte a coluna de intervalos sem jobs para um timedelta (intervalo de tempo)
    df['intervalo_sem_jobs'] = pd.to_timedelta(df['intervalo_sem_jobs'])

    #Filtra os dataframes de acordo com os intervalos máximos permitidos para cada canal e com o limite de 11h
    df = df[(df.intervalo_sem_jobs >= intervalo_max) & (df.intervalo_sem_jobs <= limite)]

    #Cria coluna para nº de jobs no intervalo
    df['jobs_no_intervalo'] = ''

    #Cria coluna para o nº de atendentes no intervalo
    df['num_xtros'] = ''

    #Varre todo o índice do dataframe resultante
    for i in df.index:

      #Atualiza a coluna de jobs nos intervalos criando novos dataframes (filtrados a partir dos limites de início e de fim dos jobs) 
      df.jobs_no_intervalo[i] = df_canal.loc[(df_canal['Action Started At Time'] > df['Action Started At Time'][i])].loc[df_canal['Action Started At Time'] <= df.fim_intervalo[i]].shape[0]

    #Filtra o dataframe resultante para os intervalos que possuem 1 ou mais jobs                                                                                                        
    df = df[df.jobs_no_intervalo > 0]
    
    #Varre todo o índice do dataframe resultante
    for i in df.index:
      #Atualiza a coluna de nº de atendentes nos intervalos criando novos dataframes (filtrados a partir dos limites de início e de fim dos jobs e removendo duplicatas dos logins) 
      df['num_xtros'] = df_canal.loc[(df_canal['Action Started At Time'] > df['Action Started At Time'][i])].loc[df_canal['Action Started At Time'] <= df.fim_intervalo[i]].drop_duplicates(subset = 'Action Actor ID').shape[0]
    
    #Cria função de nº de jobs/atendente nos intervalos
    func_jobs_por_xtronaut = lambda x, y: (x / y)  

    #Aplica a função criada para a coluna "jobs_por_xtronaut"
    df['jobs_por_xtronaut'] = func_jobs_por_xtronaut(df.jobs_no_intervalo, df.num_xtros)

    #Filtra o dataframe resultante para os intervalos que possuem 3 ou mais jobs/atendente
    df = df[df.jobs_por_xtronaut >= 3]

    #Cria uma coluna que contém o total de minutos em cada intervalo
    df['total_minutes'] = df.intervalo_sem_jobs.apply(lambda x: x.total_seconds()/60)

    #Cria coluna que contém o tempo médio para cada job ser iniciado
    df['tempo_medio_jobs'] = df.total_minutes / df.jobs_por_xtronaut

    #Converte a coluna "jobs_por_xtronaut" em tipo inteiro
    df.jobs_por_xtronaut = df.jobs_por_xtronaut.astype(int)

    #Converte a coluna "tempo_medio_jobs" em tipo inteiro
    df.tempo_medio_jobs = df.tempo_medio_jobs.astype(int)

    #Cria coluna de jobs por xtronaut (com indicativo de a cada quantos min foi iniciado um job no intervalo)
    df['jobs_por_xtronaut_detalhado'] = ''

    #Varre todo o índice do dataframe resultante
    for i in df.index:
      
      #Atualiza a coluna "jobs_por_xtronaut_detalhado"
      df.jobs_por_xtronaut_detalhado[i] = '{} ({} min)'.format(str(df.jobs_por_xtronaut[i]), str(df.tempo_medio_jobs[i]))

    #Concatena o dataframe resultante ao dataframe "atencao" (criado antes dos loopings)
    atencao = pd.concat([atencao, df])

#Substitui os termos em inglês pelos termos em português para cada canal
atencao['Job Source'] = atencao['Job Source'].replace('chat', 'Chat').replace('inbound_call', 'Phone').replace('email', 'Email')

#Reescreve os intervalos sem jobs sem a parte dos dias
atencao.intervalo_sem_jobs = atencao.intervalo_sem_jobs.apply(lambda x: str(x).replace('0 days ', ''))

#Cria coluna com a data em questão
atencao['Data'] = dia_str

#Renomeia as colunas em inglês do dataframe resultante para termos em português
atencao.rename(columns = {'Job Source' : 'canal', 'Action Actor ID': 'Login', 'Action Started At Time': 'inicio_intervalo', 'Action Final Status' : 'status'}, inplace = True)

#Unifica os dataframes "atencao" e "dimens" com base nos logins
atencao = pd.merge(atencao, dimens, how = 'inner', on = 'Login')[['Data','canal','Nome','Supervisor(a)','inicio_intervalo','fim_intervalo','intervalo_sem_jobs','jobs_por_xtronaut_detalhado']]

#Reescreve os inícios e finais de jobs apenas com as horas
atencao.inicio_intervalo = atencao.inicio_intervalo.apply(lambda x: x.time())
atencao.fim_intervalo = atencao.fim_intervalo.apply(lambda x: x.time())

#Classifica o dataframe resultante pelo início dos intervalos sem jobs
atencao.sort_values(by = 'inicio_intervalo', ascending = True, inplace = True)

#Define a planilha (sheet) a ser utilizada
intervalos_sheets = gc.open('Produtividade_02/2022')

#Define a aba (worksheet) a ser utilizada
intervalos_page = intervalos_sheets.worksheet('Intervalos_sem_jobs')

#Cria um dataframe com os dados da aba selecionada
df_intervalos = pd.DataFrame(intervalos_page.get_all_records())


#Filtra o dataframe por dia diferente do dia_str
novo_df_intervalos = df_intervalos[df_intervalos.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
atencao.columns = ['Data', 'Canal', 'Nome', 'TL', 'Início', 'Fim', 'Intervalo', 'Jobs/Xtronaut (1 job a cada...)']

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_intervalos = pd.concat([novo_df_intervalos, atencao])

#Elimina possíveis duplicatas dos intervalos
novo_df_intervalos.drop_duplicates(subset = ['Data', 'Nome', 'Intervalo'], keep = 'first', inplace = True)

#Converte os inícios e finais dos jobs, bem como os intervalos sem jobs, em string
novo_df_intervalos.Início = novo_df_intervalos.Início.apply(lambda x: str(x))
novo_df_intervalos.Fim = novo_df_intervalos.Fim.apply(lambda x: str(x))
novo_df_intervalos.Intervalo = novo_df_intervalos.Intervalo.apply(lambda x: str(x))

#Preenche os vazios (NaN) com ''
novo_df_intervalos = novo_df_intervalos.fillna('')

#Elimina linhas que tenham ficado com o campo "Data" em branco
novo_df_intervalos = novo_df_intervalos[novo_df_intervalos.Data != '']

#Atualiza a aba selecionada com o DataFrame resultante
intervalos_page.update([novo_df_intervalos.columns.values.tolist()] + novo_df_intervalos.values.tolist())

     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

{'spreadsheetId': '1Ngnweon5y_fOd6_IScQ2ru4iic7rJ-u2sDGpZ2fwkhI',
 'updatedCells': 26816,
 'updatedColumns': 8,
 'updatedRange': 'Intervalos_sem_jobs!A1:H3352',
 'updatedRows': 3352}

# IDs de jobs com possíveis gaps de WT e TS

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)']]
dimens = dimens[dimens['Login'].notna()]

#Lê o arquivo em csv com os detalhes de cada atendimento
actions = pd.read_csv('actions_done.csv', sep = ',')

#Cria um filtro para o dia em questão
filtro_dia = actions['Action Started Date'] == dia_str

#Aplica o filtro criado
actions = actions[filtro_dia]

#Filtra o dataframe para apenas jobs que tiveram o status de "finished"
actions = actions[actions['Action Final Status'] == 'finished']

#Converte as colunas de início e de fim dos jobs para datetime
actions['Action Started At Time'] = pd.to_datetime(actions['Action Started At Time'])
actions['Action Ended At Time'] = pd.to_datetime(actions['Action Ended At Time'])

#Cria uma coluna com o tempo em que o job ficou ativo
actions['tempo_job'] = actions['Action Ended At Time'] - actions['Action Started At Time']

#Remove as vírgulas da coluna de duração do job e a converte em tipo inteiro
actions['Action Duration'] = actions['Action Duration'].apply(lambda x: x.replace(',',''))
actions['Action Duration'] = actions['Action Duration'].astype(int)

#Cria filtro para a duração dos jobs do canal de chat
filtro_duracao_chat = actions['Action Duration'] > 700

#Estabelece tempo máximo para atividade dos jobs
tempo_max = pd.Timedelta ("0 days 01:30:00")

#Filtra os jobs de acordo com o tempo máximo de atividade
filtro_wt = actions.tempo_job >= tempo_max

#Cria filtro para o canal de chat
filtro_chat = actions['Job Source'] == 'chat'

#Cria um filtro para a duração dos jobs do canal de email
filtro_duracao_email = actions['Action Duration'] > 1000

#Cria filtro para o canal de email
filtro_email = actions['Job Source'] == 'email'

#Filtra o dataframe de acordo com os filtros criados anteriormente para os canais de chat e de email
IDs = actions[(filtro_duracao_chat & filtro_wt & filtro_chat) | (filtro_duracao_email & filtro_email)]

#Seleciona apenas a parte das horas das colunas de início e de fim dos jobs
IDs['Action Started At Time'] = IDs['Action Started At Time'].apply(lambda x: x.time())
IDs['Action Ended At Time'] = IDs['Action Ended At Time'].apply(lambda x: x.time())

#Renomeia as colunas com termos em português
IDs.rename(columns = {'Action Started Date' : 'Data', 'Action Actor ID' : 'Login', 'Job Source' : 'canal', 'Action Started At Time': 'inicio_job', 'Action Ended At Time' : 'fim_job', 'Action Duration' : 'duracao', 'Source ID' : 'ID' }, inplace = True)

#Troca os termos originais dos canais para Chat e Email
IDs.canal = IDs.canal.replace('chat', 'Chat').replace('email', 'Email')

#Unifica os dataframes "IDs" e "dimens" com base nos logins, e seleciona as colunas relevantes
IDs = pd.merge(IDs, dimens, how = 'inner', on = 'Login')[['Data','canal','Nome','Supervisor(a)','inicio_job','fim_job','duracao','ID']]

#Define a planilha (sheet) a ser utilizada
ids_sheets = gc.open('Produtividade_02/2022')

#Define a aba (worksheet) a ser utilizada
ids_page = ids_sheets.worksheet('Possíveis_gaps_WT_TS')

#Cria um dataframe com os dados da aba selecionada
df_ids = pd.DataFrame(ids_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_ids = df_ids[df_ids.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
IDs.columns = novo_df_ids.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_ids = pd.concat([novo_df_ids, IDs])

#Converte os inícios e os finais dos jobs para string
novo_df_ids['Início Job'] = novo_df_ids['Início Job'].apply(lambda x: str(x))
novo_df_ids['Fim Job'] = novo_df_ids['Fim Job'].apply(lambda x: str(x))
novo_df_ids = novo_df_ids.fillna('')

#Atualiza a aba selecionada com o DataFrame resultante
ids_page.update([novo_df_ids.columns.values.tolist()] + novo_df_ids.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

{'spreadsheetId': '1Ngnweon5y_fOd6_IScQ2ru4iic7rJ-u2sDGpZ2fwkhI',
 'updatedCells': 5608,
 'updatedColumns': 8,
 'updatedRange': "'Possíveis_gaps_WT_TS'!A1:H701",
 'updatedRows': 701}

# Sources IDs de jobs com nota 1

In [ ]:
#Filtra o dataframe "csat" para apenas notas 1 e seleciona colunas relevantes
IDs_csat = csat[csat['Customer Rating'] == 1][['Source ID', 'Action Actor ID','Rating Date', 'Customer Rating']]

#Define a planilha (sheet) a ser utilizada
ids_csat_sheets = gc.open('CSAT_02/2022')

#Define a aba (worksheet) a ser utilizada
ids_csat_page = ids_csat_sheets.worksheet('IDs_nota_1')

#Cria um dataframe com os dados da aba selecionada
df_ids_csat = pd.DataFrame(ids_csat_page.get_all_records())

#Elimina as linhas prévias para posterior substituição pelas atualizadas.
novo_df_ids_csat = df_ids_csat[df_ids_csat.Data == '']

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
IDs_csat.columns = novo_df_ids_csat.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_ids_csat = pd.concat([novo_df_ids_csat, IDs_csat])

#Atualiza a aba selecionada com o DataFrame resultante
ids_csat_page .update([novo_df_ids_csat.columns.values.tolist()] + novo_df_ids_csat.values.tolist())

{'spreadsheetId': '1MGyoFK7bhyn6nOm90UI4ZmbokXMXM87Lhc_b2Ft2kqA',
 'updatedCells': 5668,
 'updatedColumns': 4,
 'updatedRange': 'IDs_nota_1!A1:D1417',
 'updatedRows': 1417}

# IDs de possíveis jobs encerrados abruptamente

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)']]
dimens = dimens[dimens['Login'].notna()]

#Lê o arquivo em csv com os detalhes de cada atendimento
actions = pd.read_csv('actions_done.csv', sep = ',')

#Cria filtro para o dia em questão
filtro_dia = actions['Action Started Date'] == dia_str

#Aplica o filtro criado
actions = actions[filtro_dia]

#Filtra o dataframe para apenas jobs que tiveram o status de "finished"
actions = actions[actions['Action Final Status'] == 'finished']

#Converte as colunas de início e de fim dos jobs para datetime
actions['Action Started At Time'] = pd.to_datetime(actions['Action Started At Time'])
actions['Action Ended At Time'] = pd.to_datetime(actions['Action Ended At Time'])

#Cria uma coluna com o tempo em que o job ficou ativo
actions['tempo_job'] = actions['Action Ended At Time'] - actions['Action Started At Time']

#Remove as vírgulas da coluna de duração do job e a converte em tipo inteiro
actions['Action Duration'] = actions['Action Duration'].apply(lambda x: x.replace(',',''))
actions['Action Duration'] = actions['Action Duration'].astype(int)

#Cria filtro de duração dos jobs de chat
filtro_duracao_chat = actions['Action Duration'] < 30

#Cria filtro para os jobs de chat
filtro_chat = actions['Job Source'] == 'chat'

#Cria filtro de duração dos jobs de phone
filtro_duracao_phone = actions['Action Duration'] < 60

#Cria filtro para os jobs de phone
filtro_phone = actions['Job Source'] == 'inbound_call'

#Cria filtro de duração dos jobs de email
filtro_duracao_email = actions['Action Duration'] < 30

#Cria filtro para os jobs de email
filtro_email = actions['Job Source'] == 'email'

#Filtra o dataframe de acordo com os filtros criados anteriormente para os canais de chat, phone e email
IDs_duracao = actions[(filtro_duracao_chat & filtro_chat) | (filtro_duracao_phone & filtro_phone) | (filtro_duracao_email & filtro_email)]

#Seleciona apenas a parte das horas das colunas de início e de fim dos jobs
IDs_duracao['Action Started At Time'] = IDs_duracao['Action Started At Time'].apply(lambda x: x.time())
IDs_duracao['Action Ended At Time'] = IDs_duracao['Action Ended At Time'].apply(lambda x: x.time())

#Renomeia as colunas com termos em português
IDs_duracao.rename(columns = {'Action Started Date' : 'Data', 'Action Actor ID' : 'Login', 'Job Source' : 'canal', 'Action Started At Time': 'inicio_job', 'Action Ended At Time' : 'fim_job', 'Action Duration' : 'duracao', 'Source ID' : 'ID' }, inplace = True)

#Troca os termos originais dos canais para Chat, Phone e Email
IDs_duracao.canal = IDs_duracao.canal.replace('chat', 'Chat').replace('inbound_call', 'Phone').replace('email', 'Email')

#Unifica os dataframes "IDs_duração" e "dimens" com base nos logins, e seleciona as colunas relevantes
IDs_duracao = pd.merge(IDs_duracao, dimens, how = 'inner', on = 'Login')[['Data','canal','Login', 'Nome','Supervisor(a)','inicio_job','fim_job','duracao','ID']]

#Define a planilha (sheet) a ser utilizada
ids_duracao_sheets = gc.open('CSAT_02/2022')

#Define a aba (worksheet) a ser utilizada
ids_duracao_page = ids_duracao_sheets.worksheet('IDs_TS_baixo')

#Cria um dataframe com os dados da aba selecionada
df_ids_duracao = pd.DataFrame(ids_duracao_page.get_all_records())

#Filtra o dataframe por dia diferente do dia_str
novo_df_ids_duracao = df_ids_duracao[df_ids_duracao.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
IDs_duracao.columns = novo_df_ids_duracao.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_ids_duracao = pd.concat([novo_df_ids_duracao, IDs_duracao])

#Converte os inícios e os finais dos jobs para string
novo_df_ids_duracao.Início = novo_df_ids_duracao.Início.apply(lambda x: str(x))
novo_df_ids_duracao.Fim = novo_df_ids_duracao.Fim.apply(lambda x: str(x))

#Atualiza a aba selecionada com o DataFrame resultante
ids_duracao_page .update([novo_df_ids_duracao.columns.values.tolist()] + novo_df_ids_duracao.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

{'spreadsheetId': '1MGyoFK7bhyn6nOm90UI4ZmbokXMXM87Lhc_b2Ft2kqA',
 'updatedCells': 24642,
 'updatedColumns': 9,
 'updatedRange': 'IDs_TS_baixo!A1:I2738',
 'updatedRows': 2738}

In [ ]:
novo_df_ids_duracao

,Data,Canal,Agente,Nome,TL,Início,Fim,TS,Source ID
0,2022-02-01,Chat,bruna.cruz920.services@nubank.com.br,Bruna Camargo Da Cruz,Diogo Martins,23:43:10,23:43:37,27,pusher-chats.5ff38f10-6dd9-4323-999e-78184e9d8...
1,2022-02-01,Chat,hady.silva466.services@nubank.com.br,Hady Cleusio dos Santos Silva,Matheus Henrique de Souza,23:19:07,23:19:25,18,pusher-chats.59b9a352-d6a7-459c-a481-853ede9b8...
2,2022-02-01,Chat,bruno.gobbo567.services@nubank.com.br,Bruno Ruan De Paula Gobbo,Gabriel Leonardo da Cunha,22:43:46,22:44:13,27,pusher-chats.5dc4d098-c64b-4d06-a47d-8f7e65ec4...
3,2022-02-01,Chat,bruno.gobbo567.services@nubank.com.br,Bruno Ruan De Paula Gobbo,Gabriel Leonardo da Cunha,22:33:48,22:34:11,23,pusher-chats.615cbec0-700c-464f-8ed3-b94852634...
4,2022-02-01,Chat,bruno.gobbo567.services@nubank.com.br,Bruno Ruan De Paula Gobbo,Gabriel Leonardo da Cunha,22:33:26,22:33:48,22,pusher-chats.5dcb1f73-7c3a-4e67-9aef-3481fca7b...
...,...,...,...,...,...,...,...,...,...
236,2022-02-14,Chat,daniel.silva708.services@nubank.com.br,Daniel Paulo Dias Da Silva,Diogo Martins,01:08:52,01:09:58,28,pusher-chats.5da648fb-f776-41ca-b50b-b3318b428...
237,2022-02-14,Chat,daniel.silva708.services@nubank.com.br,Daniel Paulo Dias Da Silva,Diogo Martins,01:07:19,01:08:51,24,pusher-chats.5e027c38-9531-48b3-9c14-2accd642b...
238,2022-02-14,Phone,adriane.tupich104.services@nubank.com.br,Adriane Cristina Tupich,Diogo Martins,00:29:53,00:30:46,53,CA8302102129d45bd0a96d5d176aff7a37
239,2022-02-14,Phone,monica.souza458.services@nubank.com.br,Monica Leticia De Lima Lopes De Souza,Fernanda Thayna Costa da Silva,13:56:32,13:57:26,54,CA4d6b68611fab36233295b4b5c7354273


# IDs de jobs encerrados com TS baixo e nota 1

In [ ]:
#Renomei a coluna "Data" do "novo_df_ids_csat"
novo_df_ids_csat.rename(columns = {'Data' : 'Data_Avaliação'}, inplace = True)

#Renomei a coluna "Data" do "novo_df_ids_duracao"
novo_df_ids_duracao.rename(columns = {'Data' : 'Data_Atendimento'}, inplace = True)

#Cria um conjunto com os IDs do dataframe "novo_df_ids_csat"
a = set(novo_df_ids_csat['Source ID'])

#Cria um conjunto com os IDs do dataframe "novo_df_ids_duracao"
b = set(novo_df_ids_duracao['Source ID'])

#Estabelece uma intercecção entre os conjuntos criados
c = a.intersection(b)

#Unifica os dataframes "novo_df_ids_csat" e "novo_df_ids_duracao" com base nos logins e nos IDs e seleciona as colunas relevantes
encerramentos_abruptos = pd.merge(novo_df_ids_csat, novo_df_ids_duracao, how = 'inner', on = ['Agente', 'Source ID'])[['Data_Atendimento', 'Data_Avaliação','Canal', 'Agente', 'Nome','TL','Início','Fim','TS','Source ID', 'Nota']]

#Cria um filtro com base nos IDs em comum (conjunto C)
filtro_ids = list(c)

#Filtra o dataframe resultante com base nos IDs do conjunto C
encerramentos_abruptos = encerramentos_abruptos[encerramentos_abruptos['Source ID'].isin(filtro_ids)]

#Define a planilha (sheet) a ser utilizada
encerramentos_abruptos_sheets = gc.open('CSAT_02/2022')

#Define a aba (worksheet) a ser utilizada
encerramentos_abruptos_page = encerramentos_abruptos_sheets.worksheet('Encerramentos_abruptos_(nota_1)')

#Cria um dataframe com os dados da aba selecionada
df_encerramentos_abruptos = pd.DataFrame(encerramentos_abruptos_page.get_all_records())

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
encerramentos_abruptos.columns = df_encerramentos_abruptos.columns

#Limpa todos os dados da aba selecionada
encerramentos_abruptos_page.clear()

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_encerramentos_abruptos = pd.concat([df_encerramentos_abruptos, encerramentos_abruptos])

#Remove possíveis duplicatas de atendimentos, em caso de edição equivocada da planilha
novo_df_encerramentos_abruptos.drop_duplicates(subset = ['Nome', 'Source ID'], keep = 'first', inplace = True)

#Elimina possíveis linhas vazias do dataframe resultante
novo_df_encerramentos_abruptos = novo_df_encerramentos_abruptos[novo_df_encerramentos_abruptos.Canal != '']

#Atualiza a aba selecionada com o DataFrame resultante
encerramentos_abruptos_page.update([novo_df_encerramentos_abruptos.columns.values.tolist()] + novo_df_encerramentos_abruptos.values.tolist())

{'spreadsheetId': '1MGyoFK7bhyn6nOm90UI4ZmbokXMXM87Lhc_b2Ft2kqA',
 'updatedCells': 1463,
 'updatedColumns': 11,
 'updatedRange': "'Encerramentos_abruptos_(nota_1)'!A1:K133",
 'updatedRows': 133}

# Resultados Diários

In [ ]:
#Lê o arquivo em csv com os resultados diários
resultados = pd.read_csv('Resultados.csv')

#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)']]
dimens = dimens[dimens['Login'].notna()]

#Cria um filtro para o dia em questão
resultados = resultados[resultados['Date Date'] == dia_str]

#Converte a coluna "Net Time Spent" para string, retira as vírgulas, e converte em float (necessário pois os dados têm valores acima de 1000, possuem vírgula e não )
resultados['Net Time Spent'] = resultados['Net Time Spent'].astype(str)
resultados['Net Time Spent'] = resultados['Net Time Spent'].apply(lambda x: x.replace(',',''))
resultados['Net Time Spent'] = resultados['Net Time Spent'].astype(float)

resultados['Waiting Time'] = resultados['Waiting Time'].astype(str)
resultados['Waiting Time'] = resultados['Waiting Time'].apply(lambda x: x.replace(',',''))
resultados['Waiting Time'] = resultados['Waiting Time'].astype(float)

resultados['Recovery'] = resultados['Recovery'].astype(str)
resultados['Recovery'] = resultados['Recovery'].apply(lambda x: x.replace(',',''))
resultados['Recovery'] = resultados['Recovery'].astype(float)

#Cria filtro para o canal de chat
filtro_chat = resultados.Channel == 'chat'

#Cria limites para as métricas de chat
filtro_recovery_chat = resultados.Recovery <= 0.03
filtro_jobs_chat = resultados['Jobs Done'] <= 40
filtro_ts_chat = resultados['Net Time Spent'] >= 350
filtro_wt_chat = resultados['Waiting Time'] >= 180
filtro_skip_chat = resultados['Skip'] >= 0.05

#Filtra os resultados de chat para os limites estabelecidos anteriormente
resultados_chat = resultados[filtro_chat & (filtro_recovery_chat | filtro_jobs_chat | filtro_ts_chat| filtro_wt_chat | filtro_skip_chat)]

#Cria filtro para o canal de phone
filtro_phone = resultados.Channel == 'inbound_call'

#Cria limites para as métricas de phone
filtro_recovery_phone = resultados.Recovery <= 0.06
filtro_jobs_phone = resultados['Jobs Done'] <= 20
filtro_ts_phone = resultados['Net Time Spent'] >= 500
filtro_skip_phone = resultados['Skip'] >= 0.10

#Filtra os resultados de phone para os limites estabelecidos anteriormente
resultados_phone = resultados[filtro_phone & (filtro_recovery_phone | filtro_jobs_phone | filtro_ts_phone | filtro_skip_phone)]

#Cria filtro para o canal de email
filtro_email = resultados.Channel == 'email'

#Cria limites para as métricas de email
filtro_jobs_email = resultados['Jobs Done'] <= 50
filtro_ts_email = resultados['Net Time Spent'] >= 300
filtro_skip_email = resultados['Skip'] >= 0.06

#Filtra os resultados de email para os limites estabelecidos anteriormente
resultados_email = resultados[filtro_email & (filtro_jobs_email | filtro_ts_email | filtro_skip_email)]

#Concatena os resultados de chat, email e phone, filtrados anteriormente
resultados = pd.concat([resultados_chat, resultados_phone, resultados_email])

#Substitui os termos originais dos canais por "Chat", "Phone" e "Email"
resultados.Channel = resultados.Channel.replace('chat', 'Chat').replace('inbound_call', 'Phone').replace('email', 'Email')

#Renomeia a coluna que contém os logins
resultados.rename(columns = {'Agent' : 'Login'}, inplace = True)

#Unifica os dataframes "resultados" e "dimens" com base nos logins e seleciona as colunas relevantes
resultados = pd.merge(resultados, dimens, how = 'inner', on = 'Login')[['Channel', 'Date Date', 'Nome', 'Supervisor(a)', 'Recovery','Jobs Done', 'Skip', 'Net Time Spent',  'Waiting Time' ]]
resultados = resultados[resultados['Jobs Done'].notna()]

#Define a planilha (sheet) a ser utilizada
resultados_sheets = gc.open('Métricas diárias_02/2022')

#Define a aba (worksheet) a ser utilizada
resultados_page = resultados_sheets.worksheet('Resultados Diários')

#Cria um dataframe com os dados da aba selecionada
df_resultados = pd.DataFrame(resultados_page.get_all_records())

#Seleciona e organiza as colunas relevantes
df_resultados = df_resultados[['Canal', 'Data', 'Nome', 'TL', 'Recovery', 'Jobs', 'Skip', 'TS', 'WT']]

#Filtra o dataframe por dia diferente do dia_str
novo_df_resultados = df_resultados[df_resultados.Data != dia_str]

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
resultados.columns = novo_df_resultados.columns

#Limpa os dados da aba selecionada
resultados_page.clear()

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_resultados = pd.concat([novo_df_resultados, resultados])

#Preenche os NaN com vazios
novo_df_resultados = novo_df_resultados.fillna('')

#Elimina todas as linhas que possuem o canal como vazio (devido a erro na base de dados isso é possível)
novo_df_resultados = novo_df_resultados[novo_df_resultados.Canal != '']

#Atualiza a aba selecionada com o DataFrame resultante
resultados_page.update([novo_df_resultados.columns.values.tolist()] + novo_df_resultados.values.tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

{'spreadsheetId': '1JEetqs4coUI95kPhguucpxdcO5JyAxgqCyaDuNjF_cI',
 'updatedCells': 17352,
 'updatedColumns': 9,
 'updatedRange': "'Resultados Diários'!A1:I1928",
 'updatedRows': 1928}

#Atualiza ocorrências de métricas limites atingidas

In [ ]:
#Define a planilha (sheet) a ser utilizada
resultados_sheets = gc.open('Métricas diárias_02/2022')

#Define a primeira aba (worksheet) a ser utilizada
resultados_page = resultados_sheets.worksheet('Resultados Diários')

#Cria um dataframe com os dados da aba selecionada
df_resultados = pd.DataFrame(resultados_page.get_all_records())

#Seleciona e organiza as colunas relevantes
df_resultados = df_resultados[['Canal', 'Data', 'Nome', 'TL', 'Recovery', 'Jobs', 'Skip', 'TS', 'WT']]

#Cria um dataframe novo selecionando algumas colunas do "df_resultados"
ocorrencias = df_resultados[['Canal', 'Nome', 'TL']]

#Elimina linhas repetidas
ocorrencias.drop_duplicates(subset = ['Canal', 'Nome'], keep = 'first', inplace = True)

#Classifica o dataframe resultante
ocorrencias.sort_values(by = ['Nome', 'Canal'], inplace = True)

#Define a segunda aba (worksheet) a ser utilizada
ocorrencias_page = resultados_sheets.worksheet('Ocorrências')

#Limpa os dados da segunda aba selecionada
lista_vazio = []

for i in range(1, 500):
  lista_vazio.append(['', '', ''])

ocorrencias_page.update('A2:C500', lista_vazio)

#Atualiza a aba selecionada com o DataFrame resultanteç
ocorrencias_page.update([ocorrencias.columns.values.tolist()] + ocorrencias.values.tolist())

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


{'spreadsheetId': '1JEetqs4coUI95kPhguucpxdcO5JyAxgqCyaDuNjF_cI',
 'updatedCells': 1092,
 'updatedColumns': 3,
 'updatedRange': "'Ocorrências'!A1:C364",
 'updatedRows': 364}

# Cria DataFrame de feedbacks para Encerramentos Abruptos

In [ ]:
#Define a planilha (sheet) de CSAT a ser utilizada
csat_sheets = gc.open('CSAT_02/2022')

#Define a aba (worksheet) de Encerramentos Abruptos a ser utilizada
encerramentos_abruptos_page = csat_sheets.worksheet('Encerramentos_abruptos_(nota_1)')

#Cria um dataframe com os dados da aba de Encerramentos Abruptos
df_encerramentos_abruptos = pd.DataFrame(encerramentos_abruptos_page.get_all_records())

#Seleciona as colunas relevantes do dataframe resultante
df_encerramentos_abruptos = df_encerramentos_abruptos[['Data Avaliação', 'Canal', 'Nome', 'TL', 'Source ID']]

#Filtra o dataframe por dia igual ao dia_str
feedbacks_encerramentos_abruptos = df_encerramentos_abruptos[df_encerramentos_abruptos['Data Avaliação'] == dia_str]

#Cria uma coluna descrevendo a ocorrência em questão
feedbacks_encerramentos_abruptos['Ocorrencia'] = feedbacks_encerramentos_abruptos['Source ID'].apply(lambda x: 'Xtronaut recebeu nota 1 em atendimento de curta duração.\nPossível encerramento abrupto.\nSource ID: {}'.format(x))

#Cria uma coluna com o tipo de ocorrência
feedbacks_encerramentos_abruptos['Tipo'] = 'CSAT 1'

#Cria uma coluna com a URL da aba de Encerramentos Abruptos
feedbacks_encerramentos_abruptos['Detalhes'] = '{}/edit#gid={}'.format(csat_sheets.url, encerramentos_abruptos_page.id)

#Organiza as colunas do dataframe resultante
feedbacks_encerramentos_abruptos = feedbacks_encerramentos_abruptos[['Data Avaliação', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

#Renomeia a coluna "Data Avaliação" para padronização com os outros dataframes
feedbacks_encerramentos_abruptos.rename(columns = {'Data Avaliação': 'Data'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

# Cria DataFrame de feedbacks para Intervalos Sem Jobs

In [ ]:
#Define a planilha (sheet) de Produtividade a ser utilizada
intervalos_sheets = gc.open('Produtividade_02/2022')

#Define a aba (worksheet) de Intervalos Sem Jobs a ser utilizada
intervalos_sem_jobs_page = intervalos_sheets.worksheet('Intervalos_sem_jobs')

#Cria um dataframe com os dados da aba de Intervalos Sem Jobs
df_intervalos_sem_jobs = pd.DataFrame(intervalos_sem_jobs_page.get_all_records())

#Seleciona as colunas relevantes do dataframe resultante
df_intervalos_sem_jobs = df_intervalos_sem_jobs[['Data', 'Canal', 'Nome', 'TL','Início', 'Fim', 'Intervalo']]

#Filtra o dataframe por dia igual ao dia_str
feedbacks_intervalos_sem_jobs = df_intervalos_sem_jobs[df_intervalos_sem_jobs.Data == dia_str]

#Cria uma coluna descrevendo a ocorrência em questão
feedbacks_intervalos_sem_jobs['Ocorrencia'] = ''

#Varre o índice do dataframe resultante
for i in feedbacks_intervalos_sem_jobs.index:

  #Preenche os dados da coluna de ocorrências com base nos dados de início e finais dos jobs, bem como do intervalo em si
  feedbacks_intervalos_sem_jobs['Ocorrencia'][i] = 'Xtronaut não puxou jobs entre {} e {} ({})'.format(feedbacks_intervalos_sem_jobs['Início'][i], feedbacks_intervalos_sem_jobs['Fim'][i], feedbacks_intervalos_sem_jobs['Intervalo'][i] )  

#Cria uma coluna com o tipo de ocorrência
feedbacks_intervalos_sem_jobs['Tipo'] = 'Produtividade'

#Cria uma coluna com a URL da aba de Encerramentos Abruptos
feedbacks_intervalos_sem_jobs['Detalhes'] = '{}/edit#gid={}'.format(intervalos_sheets.url, intervalos_sem_jobs_page.id)

#Organiza as colunas do dataframe resultante
feedbacks_intervalos_sem_jobs = feedbacks_intervalos_sem_jobs[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

# Cria DataFrame de feedbacks para as métricas diárias

In [ ]:
#Define a planilha (sheet) de Métricas Diárias a ser utilizada
resultados_sheets = gc.open('Métricas diárias_02/2022')

#Define a aba (worksheet) de Resultados Diários a ser utilizada
resultados_page = resultados_sheets.worksheet('Resultados Diários')

#Cria um dataframe com os dados da aba de Resultados Diários
df_resultados = pd.DataFrame(resultados_page.get_all_records())

#Seleciona as colunas relevantes do dataframe resultante
df_resultados = df_resultados[['Canal', 'Data', 'Nome', 'TL', 'Recovery', 'Jobs', 'Skip', 'TS', 'WT']]

#Filtra o dataframe por dia igual ao dia_str
df_dia = df_resultados[df_resultados.Data == dia_str]

#Substitui os campos vazios por 0, para a devida tratativa dos dados
df_dia = df_dia.replace('', 0)


#Cria filtros para cada canal
filtro_chat = df_dia.Canal == 'Chat'
filtro_phone = df_dia.Canal == 'Phone'
filtro_email = df_dia.Canal == 'Email'

#Cria filtros de recovery para os canais de Chat e Phone
filtro_recovery_chat = df_dia.Recovery <= 0.03
filtro_recovery_phone = df_dia.Recovery <= 0.06

#Cria filtros de nº de jobs para os canais de Chat, Phone e Email
filtro_jobs_chat = df_dia.Jobs <= 40
filtro_jobs_phone = df_dia.Jobs <= 20
filtro_jobs_email = df_dia.Jobs <= 50

#Cria filtros de skip para os canais de Chat, Phone e Email
filtro_skip_chat = df_dia.Skip >= 0.05
filtro_skip_phone = df_dia.Skip >= 0.10
filtro_skip_email = df_dia.Skip >= 0.06

#Cria filtros de time spent para os canais de Chat, Phone e Email
filtro_ts_chat = df_dia.TS >= 350
filtro_ts_phone = df_dia.TS >= 500
filtro_ts_email = df_dia.TS >= 300

#Substitui os campos vazios de waiting time por 0 para a devida tratativa dos dados
df_dia.WT = df_dia.WT.replace('', 0)

#Cria filtro de waiting time para o canal de Chat
filtro_wt_chat = df_dia.WT >= 180

#Estabelece um limite máximo de ocorrências no mês para que um atendente ultrapasse cada métrica
limite = round(0.25 * pd.Timestamp(dia_str).day)

#Define a aba (worksheet) de Ocorrências a ser utilizada
ocorrencias_page = resultados_sheets.worksheet('Ocorrências')

#Cria um dataframe com os dados da aba de Ocorrências
df_ocorrencias = pd.DataFrame(ocorrencias_page.get_all_records())

#Seleciona as colunas relevantes do dataframe resultante
df_ocorrencias = df_ocorrencias[['Canal', 'Nome', 'TL', 'Recovery', 'Jobs', 'Skip', 'TS', 'WT']]

#Filtra o recovery de acordo com os limites para cada métrica criados anteriormente
df_dia_recovery = df_dia[(filtro_chat & filtro_recovery_chat) | (filtro_phone & filtro_recovery_phone)]

#Cria um dataframe baseado nos atendendentes que excederam o nº de dias limite para o recovery
df_recovery = df_ocorrencias[df_ocorrencias.Recovery > limite]

#Unifica os dataframes "df_recovery" e "df_dia_recovery" com base nos canais e nomes, e seleciona as colunas relevantes
feedback_recovery = pd.merge(df_recovery[['Canal', 'Nome']], df_dia_recovery, how = 'inner', on = ['Canal', 'Nome'])[['Data', 'Canal', 'Nome', 'TL', 'Recovery']]

#Cria uma coluna descrevendo a ocorrência em questão                                     
feedback_recovery['Ocorrencia'] = feedback_recovery['Recovery'].apply(lambda x: 'Xtronaut teve mais um dia com recovery baixo ({}%)'.format(round(x*100,1))) 

#Cria uma coluna com o tipo de ocorrência
feedback_recovery['Tipo'] = 'Recovery'

#Cria uma coluna com a URL da aba de Métricas Diárias
feedback_recovery['Detalhes'] = '{}/edit#gid={}'.format(resultados_sheets.url, resultados_page.id)

#Organiza as colunas do dataframe resultante
feedback_recovery = feedback_recovery[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

#Filtra o nº de jobs de acordo com os limites para cada métrica criados anteriormente
df_dia_jobs = df_dia[(filtro_chat & filtro_jobs_chat) | (filtro_phone & filtro_jobs_phone) | (filtro_email & filtro_jobs_email)]

#Cria um dataframe baseado nos atendendentes que excederam o nº de dias limite para o nº de jobs
df_jobs = df_ocorrencias[df_ocorrencias.Jobs > limite]

#Unifica os dataframes "df_jobs" e "df_dia_jobs" com base nos canais e nomes, e seleciona as colunas relevantes
feedback_jobs = pd.merge(df_jobs[['Canal', 'Nome']], df_dia_jobs, how = 'inner', on = ['Canal', 'Nome'])[['Data', 'Canal', 'Nome', 'TL', 'Jobs']]

#Cria uma coluna descrevendo a ocorrência em questão                                               
feedback_jobs['Ocorrencia'] = feedback_jobs['Jobs'].apply(lambda x: 'Xtronaut teve mais um dia com nº de jobs baixo ({})'.format(x)) 

#Cria uma coluna com o tipo de ocorrência
feedback_jobs['Tipo'] = 'Jobs'

#Cria uma coluna com a URL da aba de Métricas Diárias
feedback_jobs['Detalhes'] = '{}/edit#gid={}'.format(resultados_sheets.url, resultados_page.id)

#Organiza as colunas do dataframe resultante
feedback_jobs = feedback_jobs[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

#Filtra o skip de acordo com os limites para cada métrica criados anteriormente
df_dia_skip = df_dia[(filtro_chat & filtro_skip_chat) | (filtro_phone & filtro_skip_phone) | (filtro_email & filtro_skip_email)]

#Define a planilha (sheet) de Skip/Transfer a ser utilizada
skip_sheets = gc.open('Skip/Transfer_02/2022')

#Define a aba (worksheet) de Análise Skip/Transfer a ser utilizada
skip_detalhado_page = skip_sheets.worksheet('Análise Skip/Transfer')

#Cria um dataframe com os dados da aba de Análise Skip/Transfer
df_skip_detalhado = pd.DataFrame(skip_detalhado_page.get_all_records())

#Filtra o dataframe por dia igual ao dia_str
df_skip_detalhado = df_skip_detalhado[df_skip_detalhado.Data == dia_str]

#Unifica os dataframes "df_dia_skip" e "df_skip_detalhado" com base nos canais e nomes, e seleciona as colunas relevantes
feedback_skip = pd.merge(df_dia_skip, df_skip_detalhado [['Canal', 'Nome', 'Skips', 'Transfers']], how = 'inner', on = ['Canal', 'Nome'])[['Data', 'Canal', 'Nome', 'TL', 'Skip', 'Skips', 'Transfers']]

#Cria uma coluna para a ocorrência em questão
feedback_skip['Ocorrencia'] = ''

#Varre todo o índice do dataframe resultante
for i in feedback_skip.index:

  #Cria condição para caso o canal em questão ser Phone
  if feedback_skip['Canal'][i] == 'Phone':

    #Preenche os dados da coluna de ocorrências com base nos dados de total de jobs jobs transferidos, bem como da percentagem que estes representam
    feedback_skip['Ocorrencia'][i] = 'Xtronaut teve um dia com skip alto.\nTotal de {} jobs transferidos ({}%)'.format(feedback_skip.Transfers[i], round(100*feedback_skip.Skip[i],1))

  #Caso o canal em questão não seja Phone
  else:
  
    #Preenche os dados da coluna de ocorrências com base nos dados de total de jobs jobs skipados, bem como da percentagem que estes representam
    feedback_skip['Ocorrencia'][i] = 'Xtronaut teve um dia com skip alto.\nTotal de {} jobs skipados ({}%)'.format(feedback_skip.Skips[i], round(100*feedback_skip.Skip[i],1))

#Cria uma coluna com o tipo de ocorrência
feedback_skip['Tipo'] = 'Skip (%)'

#Cria uma coluna com a URL da aba de Métricas diárias
feedback_skip['Detalhes'] = '{}/edit#gid={}'.format(resultados_sheets.url, resultados_page.id)

#Organiza as colunas do dataframe resultante
feedback_skip = feedback_skip[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

#Define a aba (worksheet) de IDs Repetidos a ser utilizada
ids_repetidos_page = skip_sheets.worksheet('IDs_repetidos')

#Cria um dataframe com os dados da aba de IDs Repetidos
df_ids_repetidos = pd.DataFrame(ids_repetidos_page.get_all_records())

#Filtra o dataframe por dia igual ao dia_str
feedback_ids_repetidos = df_ids_repetidos[df_ids_repetidos.Data == dia_str]

#Cria uma coluna para a ocorrência em questão
feedback_ids_repetidos['Ocorrencia'] = ''

#Varre todo o índice do dataframe resultante
for i in feedback_ids_repetidos.index:

  #Cria condição para caso o status em questão ser "transferred"
  if feedback_ids_repetidos.Status[i] == 'transferred':

    #Preenche os dados da coluna de ocorrências com base nos IDs
    feedback_ids_repetidos['Ocorrencia'][i] = 'Xtronaut transferiu job para outro squad que retornou.\nÉ possível que a tratativa seja realizada por collections.\nSource ID: {}'.format(feedback_ids_repetidos['Source ID'][i])

  #Para o caso de o status em questão não ser "transferred"
  else:

    #Preenche os dados da coluna de ocorrências com base nos IDs
    feedback_ids_repetidos['Ocorrencia'] [i] = 'Xtronaut skipou job para collections e este retornou.\nÉ possível que a tratativa tenha sido feita incorretamente.\nSource ID: {}'.format(feedback_ids_repetidos['Source ID'][i])

#Cria uma coluna com o tipo de ocorrência
feedback_ids_repetidos['Tipo'] = feedback_ids_repetidos.Status.apply(lambda x: 'Skip' if x == 'skipped' else 'Transfer')

#Cria uma coluna com a URL da aba de Encerramentos Abruptos
feedback_ids_repetidos['Detalhes'] = '{}/edit#gid={}'.format(skip_sheets.url, ids_repetidos_page.id)

#Organiza as colunas do dataframe resultante
feedback_ids_repetidos = feedback_ids_repetidos[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

#Filtra o time spent de acordo com os limites para cada métrica criados anteriormente
df_dia_ts = df_dia[(filtro_chat & filtro_ts_chat) | (filtro_phone & filtro_ts_phone) | (filtro_email & filtro_ts_email)]

#Cria um dataframe baseado nos atendendentes que excederam o nº de dias limite para o time spent
df_ts = df_ocorrencias[df_ocorrencias.TS > limite]

#Unifica os dataframes "df_ts" e "df_dia_ts" com base nos canais e nomes, e seleciona as colunas relevantes
feedback_ts = pd.merge(df_ts[['Canal', 'Nome']], df_dia_ts, how = 'inner', on = ['Canal', 'Nome'])[['Data', 'Canal', 'Nome', 'TL', 'TS']]
                                               
#Cria uma coluna descrevendo a ocorrência em questão
feedback_ts['Ocorrencia'] = feedback_ts['TS'].apply(lambda x: 'Xtronaut teve mais um dia com TS alto ({})'.format(x)) 

#Cria uma coluna com o tipo de ocorrência
feedback_ts['Tipo'] = 'TS'

#Cria uma coluna com a URL da aba de Métricas Diárias
feedback_ts['Detalhes'] = '{}/edit#gid={}'.format(resultados_sheets.url, resultados_page.id)

#Organiza as colunas do dataframe resultante
feedback_ts = feedback_ts[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

#Filtra o waiting time de acordo com os limites para cada métrica criados anteriormente
df_dia_wt = df_dia[(filtro_chat & filtro_wt_chat)]

#Cria um dataframe baseado nos atendendentes que excederam o nº de dias limite para o waiting time
df_wt = df_ocorrencias[df_ocorrencias.WT > limite]

#Unifica os dataframes "df_wt" e "df_dia_wt" com base nos canais e nomes, e seleciona as colunas relevantes
feedback_wt = pd.merge(df_wt[['Canal', 'Nome']], df_dia_wt, how = 'inner', on = ['Canal', 'Nome'])[['Data', 'Canal', 'Nome', 'TL', 'WT']]
                                           
#Cria uma coluna descrevendo a ocorrência em questão
feedback_wt['Ocorrencia'] = feedback_wt['WT'].apply(lambda x: 'Xtronaut teve mais um dia com WT alto ({})'.format(x)) 

#Cria uma coluna com o tipo de ocorrência
feedback_wt['Tipo'] = 'WT'

#Cria uma coluna com a URL da aba de Métricas Diárias
feedback_wt['Detalhes'] = '{}/edit#gid={}'.format(resultados_sheets.url, resultados_page.id)

#Organiza as colunas do dataframe resultante
feedback_wt = feedback_wt[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_lau

# Cria DataFrame de feedbacks para as ADH Iniciados e Finalizados

In [ ]:
#Lê o arquivo de Excel com o dimensionamento da operação, selecionando colunas relevantes
dimens = pd.read_excel('Dimensionamento.xlsx', sheet_name = 'Dimensionamento')
dimens = dimens [['Login', 'Nome', 'Supervisor(a)', 'Jornada']]
dimens = dimens[dimens['Login'].notna()]

#Define a planilha (sheet) de Aderência a ser utilizada
adh_sheets = gc.open('Hora a hora/ADH_02/2022')

#Define a aba (worksheet) de Produtividada Hora a Hora a ser utilizada
produtividade_page = adh_sheets.worksheet('Produtividade_hora_a_hora')

#Cria um dataframe com os dados da aba de Produtividada Hora a Hora
df_produtividade = pd.DataFrame(produtividade_page.get_all_records())

#Filtra o dataframe por dia igual ao dia_str
df_produtividade = df_produtividade[df_produtividade.Data == dia_str]

#Unifica os dataframes "df_produtividade" e "dimens" com base nos nomes e seleciona as colunas relevantes
feedback_adh_iniciados = pd.merge(df_produtividade, dimens[['Nome', 'Jornada']], how = 'inner', on = 'Nome')[['Data', 'Canal', 'Nome', 'Jornada', 'Supervisor(a)', 'Iniciados', 'Finalizados']]

#Cria um filtro para levar em consideração apenas linhas que contenham pelo menos 1 job iniciado
filtro_iniciados = feedback_adh_iniciados.Iniciados > 0

#Filtra o dataframe com o filtro criado anteriormente
feedback_adh_iniciados = feedback_adh_iniciados[filtro_iniciados]

#Agrupa os dados da coluna "Iniciados" com base nos campos 'Data', 'Canal','Nome', 'Jornada', 'Supervisor(a)' e cria um dataframe com os dados resultantes
feedback_adh_iniciados = pd.DataFrame(feedback_adh_iniciados.groupby(['Data', 'Canal','Nome', 'Jornada', 'Supervisor(a)'])['Iniciados'].count()).reset_index()

#Cria um filtro para levar em consideração linhas que contenham no máximo 6 jobs iniciados
filtro_6x1_iniciados = feedback_adh_iniciados.Iniciados < 7

#Cria um filtro para a jornada 6X1
filtro_6x1 = feedback_adh_iniciados.Jornada == '6X1'

#Cria um filtro para levar em consideração linhas que contenham no máximo 8 jobs iniciados
filtro_5x2_iniciados = feedback_adh_iniciados.Iniciados < 9

#Cria um filtro para a jornada 5X2
filtro_5x2 = feedback_adh_iniciados.Jornada == '5X2'

#Aplica os filtros criados no dataframe resultante
feedback_adh_iniciados = feedback_adh_iniciados[(filtro_6x1 & filtro_6x1_iniciados) | (filtro_5x2 & filtro_5x2_iniciados)]

#Cria uma coluna com a ocorrência em questão
feedback_adh_iniciados['Ocorrencia'] = ''

#Varre o índice do dataframe resultante
for i in feedback_adh_iniciados.index:

  #Cria uma condição para caso a jornada em questão seja 5X2
  if feedback_adh_iniciados.Jornada[i] == '5X2':

    #Preenche as ocorrências com base na aderência de jobs iniciados para a jornada 5X2
    feedback_adh_iniciados.Ocorrencia[i] = 'Xtronaut não iniciou jobs em todas as 9 horas do seu expediente.\n(Total de {} para a ADH)'.format(feedback_adh_iniciados.Iniciados[i]) 
  
  #Caso não se enquadre na condição
  else:

    #Preenche as ocorrências com base na aderência de jobs iniciados para a jornada 6x1
    feedback_adh_iniciados.Ocorrencia[i] = 'Xtronaut não iniciou jobs em todas as 7 horas do seu expediente.\n(Total de {} para a ADH)'.format(feedback_adh_iniciados.Iniciados[i]) 

#Cria uma coluna com o tipo de ocorrência
feedback_adh_iniciados['Tipo'] = 'ADH - I'

#Cria uma coluna com a URL da aba de ADH Iniciados
feedback_adh_iniciados['Detalhes'] = '{}/edit#gid={}'.format(adh_sheets.url, ahd_iniciados_page.id)

#Renomeia a coluna de "Supervisor(a)"
feedback_adh_iniciados.rename(columns = {'Supervisor(a)' : 'TL'}, inplace = True)

#Organiza as colunas do dataframe resultante
feedback_adh_iniciados = feedback_adh_iniciados[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

#Unifica os dataframes "df_produtividade" e "dimens" com base nos nomes e seleciona as colunas relevantes
feedback_adh_finalizados = pd.merge(df_produtividade, dimens[['Nome', 'Jornada']], how = 'inner', on = 'Nome')[['Data', 'Canal', 'Nome', 'Jornada', 'Supervisor(a)', 'Iniciados', 'Finalizados']]

#Cria um filtro para levar em consideração apenas linhas que contenham pelo menos 1 job finalizado
filtro_finalizados = feedback_adh_finalizados.Finalizados > 0

#Filtra o dataframe com o filtro criado anteriormente
feedback_adh_finalizados = feedback_adh_finalizados[filtro_finalizados]

#Agrupa os dados da coluna "Finalizados" com base nos campos 'Data', 'Canal','Nome', 'Jornada', 'Supervisor(a)' e cria um dataframe com os dados resultantes
feedback_adh_finalizados = pd.DataFrame(feedback_adh_finalizados.groupby(['Data', 'Canal','Nome', 'Jornada', 'Supervisor(a)'])['Finalizados'].count()).reset_index()

#Cria um filtro para levar em consideração linhas que contenham no máximo 6 jobs finalizados
filtro_6x1_finalizados = feedback_adh_finalizados.Finalizados < 7

#Cria um filtro para a jornada 6X1
filtro_6x1 = feedback_adh_finalizados.Jornada == '6X1'

#Cria um filtro para levar em consideração linhas que contenham no máximo 8 jobs finalizados
filtro_5x2_finalizados = feedback_adh_finalizados.Finalizados < 9

#Cria um filtro para a jornada 5X2
filtro_5x2 = feedback_adh_finalizados.Jornada == '5X2'

#Aplica os filtros criados no dataframe resultante
feedback_adh_finalizados = feedback_adh_finalizados[(filtro_6x1 & filtro_6x1_finalizados) | (filtro_5x2 & filtro_5x2_finalizados)]

#Cria uma coluna com a ocorrência em questão
feedback_adh_finalizados['Ocorrencia'] = ''

#Varre o índice do dataframe resultante
for i in feedback_adh_finalizados.index:

  #Cria uma condição para caso a jornada em questão seja 5X2
  if feedback_adh_finalizados.Jornada[i] == '5X2':

    #Preenche as ocorrências com base na aderência de jobs iniciados para a jornada 5X2
    feedback_adh_finalizados.Ocorrencia[i] = 'Xtronaut não finalizou jobs em todas as 9 horas do seu expediente.\n(Total de {} para a ADH)'.format(feedback_adh_finalizados.Finalizados[i]) 
  
  #Caso não se enquadre na condição
  else:

    #Preenche as ocorrências com base na aderência de jobs iniciados para a jornada 6x1
    feedback_adh_finalizados.Ocorrencia[i] = 'Xtronaut não finalizou jobs em todas as 7 horas do seu expediente.\n(Total de {} para a ADH)'.format(feedback_adh_finalizados.Finalizados[i]) 

#Cria uma coluna com o tipo de ocorrência
feedback_adh_finalizados['Tipo'] = 'ADH - F'

#Cria uma coluna com a URL da aba de ADH Finalizados
feedback_adh_finalizados['Detalhes'] = '{}/edit#gid={}'.format(adh_sheets.url, ahd_finalizados_page.id)

#Renomeia a coluna de "Supervisor(a)"
feedback_adh_finalizados.rename(columns = {'Supervisor(a)' : 'TL'}, inplace = True)

#Organiza as colunas do dataframe resultante
feedback_adh_finalizados = feedback_adh_finalizados[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Detalhes']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to

# Atualiza planilha de feedbacks

In [ ]:
#Concatena todos os dataframes de feedbacks criados até o momento
feedback_geral = pd.concat([feedbacks_encerramentos_abruptos, feedbacks_intervalos_sem_jobs, feedback_recovery, feedback_jobs, feedback_skip, feedback_ids_repetidos, feedback_ts, feedback_wt, feedback_adh_iniciados, feedback_adh_finalizados])

#Inclui uma coluna para justificativa
feedback_geral['Justificativa'] = ''

#Inclui uma coluna para ação a ser tomada
feedback_geral['Ação'] = ''

#Organiza as colunas do dataframe resultante
feedback_geral = feedback_geral[['Data', 'Canal', 'Nome', 'TL', 'Tipo', 'Ocorrencia', 'Justificativa', 'Ação', 'Detalhes']]

#Define a planilha (sheet) de Feedbacks a ser utilizada
feedbacks_coll_sheets = gc.open('Feedbacks_Collections_02/2022')

#Define a aba (worksheet) de Feedbacks a ser utilizada
feedbacks_page = feedbacks_coll_sheets.worksheet('Feedbacks')

#Cria um dataframe com os dados da aba de Feedbacks
df_feedbacks = pd.DataFrame(feedbacks_page.get_all_records())

#Filtra o dataframe por dias diferentes do dia_str
novo_df_feedbacks = df_feedbacks[df_feedbacks.Data != dia_str]

#Limpa os dados da aba selecionada
feedbacks_page.clear()

#Iguala as colunas do dataframe gerado inicialmente com as colunas da aba selecionada
feedback_geral.columns = df_feedbacks.columns

#Concatena o dataframe filtrado da aba selecionada com o dataframe do dia em questão
novo_df_feedbacks = pd.concat([novo_df_feedbacks, feedback_geral])

#Preenche os NaN com vazios
novo_df_feedbacks = novo_df_feedbacks.fillna('')

#Elimina possíveis linhas com Canal vazio
novo_df_feedbacks = novo_df_feedbacks[novo_df_feedbacks.Canal != '']

#Classifica o dataframe resultante
novo_df_feedbacks.sort_values(by = ['Data', 'TL', 'Nome', 'Tipo'], inplace = True)

#Atualiza a aba selecionada com o DataFrame resultante
feedbacks_page.update([novo_df_feedbacks.columns.values.tolist()] + novo_df_feedbacks.values.tolist())


{'spreadsheetId': '19qZVxiDh2vVeeAbxm73isdg1dxIbo9u2wWRWTw-fVzo',
 'updatedCells': 80343,
 'updatedColumns': 9,
 'updatedRange': 'Feedbacks!A1:I8927',
 'updatedRows': 8927}